In this notebok, you will find routines to load data from AMDA and turn them in dataset non centered around an existing flux rope

In [1]:
import pandas as pd
import sys
sys.path.append('../../identification/')

import event as evt
import numpy as np
import datetime

import speasy as spz
spz.config.proxy_url.set('http://sciqlop.lpp.polytechnique.fr/cache')
spz.config.proxy_enabled.set('True')
from speasy.inventory.data_tree import amda

## MMS

In [2]:
df = pd.read_csv('FR_MMS.txt', sep='\t').dropna()
df['Time'] = pd.to_timedelta(df['Time'])
df['Date'] = pd.to_datetime(df['Date'])
df['Unnamed: 4'] = pd.to_timedelta(df['Unnamed: 4'])
evtList = [evt.Event(df['Date'][i]+df['Time'][i], df['Date'][i]+df['Unnamed: 4'][i])for i in range(0, len(df))]

In [3]:
date_range = pd.date_range(datetime.datetime(2015, 9, 1),
                           datetime.datetime(2017,2,8),
                           freq='0.1S')
y_index = pd.Series(index=date_range, data=0)

for elt in evtList:
    y_index[elt.begin:elt.end]=1

In [4]:
fr_timeseries = []
y_label = []
delta = datetime.timedelta(seconds=0)

for i, elt in enumerate(evtList):
    
    
    duration = elt.end-elt.begin
    
    prod_duration = np.random.uniform(0.15, 0.5)
    prop_left = np.random.uniform(0, 1-prod_duration)
    start = elt.begin-duration*prop_left/prod_duration
    end = start+duration/prod_duration
    
    print((end-elt.end))
    
    data = spz.get_data(amda.Parameters.MMS.MMS1.FGM.mms1_fgm_srvy.mms1_b_gsm,start_time = start, stop_time=end)
    df_c = data.to_dataframe(datetime_index=True).dropna()
    freq = (df_c.index[-1]-df_c.index[0])/2079
    df_c = df_c.resample(freq).mean().interpolate()
    fr_timeseries.append(df_c)
    
    serie = y_index[start:end].copy()
    freq = (serie.index[-1]-serie.index[0])/2079
    serie = serie.resample(freq).mean().interpolate()
    y_label.append(serie)
    
np.save('/DATA/nguyen/timeSeries_interval/MMS_FR_non_centered.npy', np.asarray(fr_timeseries))
np.save('/DATA/nguyen/timeSeries_interval/MMS_label_non_centered.npy', np.asarray(y_label))

0 days 00:00:39.259628366
0 days 00:00:11.895857406
0 days 00:01:30.078182299
0 days 00:00:26.890786812
0 days 00:00:06.421942700
0 days 00:00:38.629804006
0 days 00:00:17.495134160
0 days 00:00:24.560561190
0 days 00:00:08.903534870
0 days 00:00:00.552117340
0 days 00:00:29.325868444
0 days 00:00:08.176716282
0 days 00:00:01.735826704
0 days 00:00:00.432872343
0 days 00:00:29.173534950
0 days 00:00:17.667227878
0 days 00:00:23.336464295
0 days 00:00:52.349183403
0 days 00:01:05.438816042
0 days 00:00:27.425267802
0 days 00:00:38.777500185
0 days 00:00:27.925790168
0 days 00:00:00.471291093
0 days 00:01:10.619412186
0 days 00:00:20.143363053
0 days 00:00:38.840497732
0 days 00:00:15.204955892
0 days 00:00:52.127920008
0 days 00:01:17.579037025
0 days 00:00:14.636872617
0 days 00:00:09.182464205
0 days 00:01:03.802036373
0 days 00:00:18.475114126
0 days 00:00:39.816034902
0 days 00:00:24.313273770
0 days 00:00:55.658457066
0 days 00:00:20.627193127
0 days 00:00:35.668315214
0 days 00:00

In [8]:
fr_timeseries = []
list_MSH_intervals = evt.read_csv('MSH_intervals.csv', header=0)
delta = datetime.timedelta(seconds=0)

for i, elt in enumerate(list_MSH_intervals):
    print(i, end='\r')
    data = spz.get_data(amda.Parameters.MMS.MMS1.FGM.mms1_fgm_srvy.mms1_b_gsm,start_time = elt.begin, stop_time=elt.end)
    df_c = data.to_dataframe(datetime_index=True).dropna()
    freq = (df_c.index[-1]-df_c.index[0])/2079
    df_c = df_c.resample(freq).mean().interpolate()
    fr_timeseries.append(df_c)
np.save('/DATA/nguyen/timeSeries_interval/MSH_MMS.npy', np.asarray(fr_timeseries))

## WIND

In [2]:
list_icmes = spz.get_data(amda.Catalogs.SharedCatalogs.SOLAR_WIND.listOfICMEs_Nguyen)
evtList = [evt.Event(list_icmes[i].start_time, list_icmes[i].stop_time)for i in range(0, len(list_icmes))]



In [3]:

no_ICME_interval = []
i = 0

while i<300:
    print(i, end='\r')
    year = np.random.randint(2000, 2016)
    month = np.random.randint(1, 13)
    day = np.random.randint(1, 29)
    hour = np.random.randint(1, 24)
    
    date = datetime.datetime(year, month, day, hour, tzinfo=datetime.timezone.utc)
    delta = datetime.timedelta(hours=10)
    interval = evt.Event(date, date+delta)
    if max(evt.overlapWithList(interval, evtList))==datetime.timedelta(0):
        data = spz.get_data(amda.Parameters.Wind.MFI.wnd_mfi_kp.wnd_b,start_time = interval.begin, stop_time=interval.end)
        df_c = data.to_dataframe(datetime_index=True).dropna()
        if len(df_c)>0:
            i += 1
            freq = (df_c.index[-1]-df_c.index[0])/2079
            df_c = df_c.resample(freq).mean().interpolate()
            no_ICME_interval.append(df_c)

In [4]:
np.save('/DATA/nguyen/timeSeries_interval/Wind_no_FR.npy', np.asarray(no_ICME_interval))

In [5]:
date_range = pd.date_range(datetime.datetime(1997, 6, 1),
                           datetime.datetime(2016,1,1),
                           freq='10S')
y_index = pd.Series(index=date_range, data=0)

for elt in evtList:
    y_index[elt.begin:elt.end]=1

/tmp/ipykernel_29203/3120584296.py:7: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  y_index[elt.begin:elt.end]=1


In [8]:
wind_timeseries = []
y_label = []
for i, elt in enumerate(evtList):
    print(i, end='\r')
    duration = elt.end-elt.begin
    
    prod_duration = np.random.uniform(0.15, 0.5)
    prop_left = np.random.uniform(0, 1-prod_duration)
    start = elt.begin-duration*prop_left/prod_duration
    end = start+duration/prod_duration
    
    data = spz.get_data(amda.Parameters.Wind.MFI.wnd_mfi_kp.wnd_b,start_time = start, stop_time=end)
    df = data.to_dataframe(datetime_index=True).dropna()
    freq = (df.index[-1]-df.index[0])/2079
    df = df.resample(freq).mean().interpolate()
    
    wind_timeseries.append(df)
    
    serie = y_index[start:end].copy()
    freq = (serie.index[-1]-serie.index[0])/2079
    serie = serie.resample(freq).mean().interpolate()
    y_label.append(serie)
    


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  se

...

/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  se

/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  se

/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  se

/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()


/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  serie = y_index[start:end].copy()
/tmp/ipykernel_29203/1153396954.py:19: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  se

In [10]:
np.save('/DATA/nguyen/timeSeries_interval/Wind_FR.npy', np.asarray(wind_timeseries))

In [11]:
np.save('/DATA/nguyen/timeSeries_interval/Wind_FR_label.npy', np.asarray(y_label))

In [65]:
seum = np.load('/DATA/nguyen/timeSeries_interval/Wind_FR.npy', allow_pickle=True)

## Cluster 1

In [4]:
df = pd.read_csv('Custer_FTE_full.txt', sep='\t').dropna()
df['tbegin'] = pd.to_datetime(df['tbegin'])

df['tend'] = pd.to_datetime(df['tend'])
evtList = [evt.Event(df['tbegin'][i], df['tend'][i])for i in range(0, len(df))]

In [5]:
date_range = pd.date_range(datetime.datetime(2002, 11, 1),
                           datetime.datetime(2003,7,1),
                           freq='0.1S')
y_index = pd.Series(index=date_range, data=0)

for elt in evtList:
    y_index[elt.begin:elt.end]=1

In [6]:
cluster_timeseries = []
y_label = []
for i, elt in enumerate(evtList):
    duration = elt.end-elt.begin

    prod_duration = np.random.uniform(0.15, 0.5)
    prop_left = np.random.uniform(0, 1-prod_duration)
    start = elt.begin-duration*prop_left/prod_duration
    end = start+duration/prod_duration
    print((end-elt.end))
    if df.sc[i]=='c1':
        data = spz.get_data(amda.Parameters.Cluster.Cluster1.FGM.clust1_fgm_high.c1_bgsm_5vps,start_time = start, stop_time=end)
    else:
        data = spz.get_data(amda.Parameters.Cluster.Cluster3.FGM.clust3_fgm_high.c3_bgsm_5vps,start_time = start, stop_time=end)

    df_c = data.to_dataframe(datetime_index=True).dropna()
    freq = (df_c.index[-1]-df_c.index[0])/2079
    df_c = df_c.resample(freq).mean().interpolate()
    
    cluster_timeseries.append(df_c)
    
    serie = y_index[start:end].copy()
    freq = (serie.index[-1]-serie.index[0])/2079
    serie = serie.resample(freq).mean().interpolate()
    y_label.append(serie)
    
np.save('/DATA/nguyen/timeSeries_interval/Cluster_FR_non_centered.npy', np.asarray(cluster_timeseries))
np.save('/DATA/nguyen/timeSeries_interval/Cluster_FR_label_non_centered.npy', np.asarray(y_label))

0 days 00:00:18.461999212
0 days 00:01:01.188998063
0 days 00:00:12.367454015
0 days 00:00:00.559471665
0 days 00:00:23.601174944
0 days 00:00:43.680295744
0 days 00:02:32.346569279
0 days 00:00:17.099574468
0 days 00:00:01.378479643
0 days 00:02:54.486999080
0 days 00:00:10.538751144
0 days 00:03:19.700940717
0 days 00:01:04.345707715
0 days 00:01:06.250727748
0 days 00:00:38.651210686
0 days 00:01:33.449748720
0 days 00:00:29.921972146
0 days 00:00:15.939070774
0 days 00:02:13.329773090
0 days 00:00:01.417268140
0 days 00:00:11.788850686
0 days 00:01:25.556364974
0 days 00:00:53.130752826
0 days 00:01:23.563805861
0 days 00:00:06.435102411
0 days 00:01:40.156119098
0 days 00:03:39.000163976
0 days 00:01:46.489700142
0 days 00:02:13.022200020
0 days 00:00:50.580873789
0 days 00:02:02.493592712
0 days 00:01:23.546702771
0 days 00:00:35.918920039
0 days 00:01:38.100620755
0 days 00:00:25.654280924
0 days 00:01:59.267126537
0 days 00:00:50.055258846
0 days 00:00:36.045564004
0 days 00:00

In [6]:
seum = pd.read_pickle('/DATA/michotte/MSP_data/5S_V1/data_gsm.pkl')

In [7]:
seum = seum[seum.sat=='C1']

In [8]:
seum = seum[seum.time<datetime.datetime(2003,6,27)]
seum = seum[seum.time>datetime.datetime(2002,10,1)]


In [9]:
indices = np.random.randint(13, len(seum), 50)
list_MSH_intervals = []

for i in indices:
    list_MSH_intervals.append(evt.Event(seum.time[seum.index[i]]-datetime.timedelta(seconds=65), seum.time[seum.index[i]]+datetime.timedelta(seconds=65)))
    
    
[max(evt.overlapWithList(x, list_MSH_intervals)) for x in evtList]

[datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timedelta(0),
 datetime.timede

In [10]:
fr_timeseries = []

delta = datetime.timedelta(seconds=0)

for i, elt in enumerate(list_MSH_intervals):
    print(i, end='\r')
    data = spz.get_data(amda.Parameters.Cluster.Cluster1.FGM.clust1_fgm_high.c1_bgsm_5vps,start_time = elt.begin, stop_time=elt.end)
    df_c = data.to_dataframe(datetime_index=True).dropna()
    freq = (df_c.index[-1]-df_c.index[0])/2079
    df_c = df_c.resample(freq).mean().interpolate()
    fr_timeseries.append(df_c)
np.save('/DATA/nguyen/timeSeries_interval/MSP_Cluster.npy', np.asarray(fr_timeseries))

FileNotFoundError: [Errno 2] No such file or directory: '/DATA/nguyen//DATA/nguyen/timeSeries_interval/MSP_Cluster.npy'

In [13]:
np.save('/DATA/nguyen/timeSeries_interval/MSP_Cluster.npy', np.asarray(fr_timeseries))